In [1]:
# %load retinanet_train.py
#!/usr/bin/env python

# In[1]:

print("Start importing package...")
import time
import os
import copy
import argparse
import pdb
import collections
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision

import model
import mymodel
from anchors import Anchors
import losses
from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from torch.utils.data import Dataset, DataLoader
from mydataloader import MyDataset,MyAugmenter
import coco_eval
import csv_eval
import visdom
vis = visdom.Visdom(env='retinanet_seresnextTest')

assert torch.__version__.split('.')[1] == '4'

print('CUDA available: {}'.format(torch.cuda.is_available()))
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#参数
NAME="RSNA"
DATA_PATH = "/data/krf/dataset"
CSV_TRAINS = [DATA_PATH + "/test_csv_train0.csv",DATA_PATH + "/test_csv_train1.csv",DATA_PATH + "/test_csv_train2.csv",DATA_PATH + "/test_csv_train3.csv"]
CSV_VALS = [DATA_PATH + "/test_csv_val0.csv",DATA_PATH + "/test_csv_val1.csv",DATA_PATH + "/test_csv_val2.csv",DATA_PATH + "/test_csv_val3.csv"]
CSV_CLASSES = DATA_PATH + "/test_classes.csv"
DEPTH = 50
EPOCHS = 20
BATCH_SIZE=2
VAL_SIZE = 30
#TRAIN_SIZE = 100
#数据预处理
import csv
import random
from tqdm import tqdm

Start importing package...
CUDA available: True


In [2]:
# with open(DATA_PATH+"/stage_1_train_labels.csv") as f:
#     reader = csv.reader(f)
#     rows=[row for row in  reader]
#     rows = rows[1:]
#     random.shuffle(rows)
#     for row in rows:
#         row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
#         if row[1] == '' and row[2] == '' and row[3] == '' and row[4] == '':
#             row[5] = ''
#         else:
#             row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
#             row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
#     val_rows = rows[:VAL_SIZE]
#     train_rows = rows[VAL_SIZE:]
#     print(len(val_rows),len(train_rows))
#     with open(CSV_TRAIN,'w') as f2:
#         write = csv.writer(f2)
#         write.writerows(train_rows)
#         print("csv_train 写入完毕")
#     with open(CSV_VAL,'w') as f3:
#         write = csv.writer(f3)
#         write.writerows(val_rows)
#         print("csv_val 写入完毕")

# with open(CSV_CLASSES,'w') as f:
#     write = csv.writer(f)
    
#     row = ['0','0']
#     write.writerow(row)
#     row = ['1','1']
#     write.writerow(row)
#     row = ['2','2']
#     write.writerow(row)
#     print("csv_classes 写入完毕")



# # #每次跑这个函数之前需要先删除之前的

# df = pd.read_csv(DATA_PATH+"/stage_1_train_labels.csv")
# ddf = pd.read_csv(DATA_PATH+'/stage_1_detailed_class_info.csv')
# train_images = os.listdir(DATA_PATH+"/stage_1_train_images")[:500]

# random.shuffle(train_images)#打乱图片顺序
# count = 0
# pos_cnt_train = [0,0,0,0]
# pos_cnt_val = [0,0,0,0]
# class_dict = {'Normal':0,'Lung Opacity':1,'No Lung Opacity / Not Normal':2}
# for img_name in tqdm(train_images):
#     results = df[df['patientId']==img_name.split('.')[0]].values
#     detail = ddf[df['patientId']==img_name.split('.')[0]].values
#     for row in results:
#         row[0] = DATA_PATH+"/stage_1_train_images/"+row[0]+".dcm"
#         if row[5] == 1:
#             pos_cnt_val[count % 4] += 1
#         if row[1] >= 0 and row[1] <= 1024:
#             row[3] = str(float(row[1]) + float(row[3]))# x2 = x1 + w 
#             row[4] = str(float(row[2]) + float(row[4]))# y2 = y1 + h
#         else:
#             row[1] = ''
#             row[2] = ''
#             row[3] = ''
#             row[4] = ''
#             row[5] = class_dict[detail[0,1]]
#         with open(CSV_VALS[count % 4],'a') as f:
#             write = csv.writer(f)
#             write.writerow(row)
#         for i in range(4):
#             if count % 4 == i:
#                 continue
#             else:
#                 with open(CSV_TRAINS[count % 4],'a') as f:
#                     write = csv.writer(f)
#                     write.writerow(row) 
#     count += 1
# print(pos_cnt_train,pos_cnt_val)  

In [3]:

print("Loading data...")

#%time
#制作数据loader
dataset_train = []
dataset_val = []

for i in range(4): 
    dataset_train.append(MyDataset(train_file=CSV_TRAINS[i], class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), MyAugmenter(), Resizer()])))
    dataset_val.append(MyDataset(train_file=CSV_VALS[i], class_list=CSV_CLASSES, transform=transforms.Compose([Normalizer(), Resizer()])))

#每次的sampler的参数：来源、batchsize、是否抛弃最后一层？？？

# num_workers 同时工作的组？collater:校验用的吧
dataloader_train = []
dataloader_val = []

for i in range(4):
    sampler = AspectRatioBasedSampler(dataset_train[i], batch_size=BATCH_SIZE, drop_last=False)
    sampler_val = AspectRatioBasedSampler(dataset_val[i], batch_size=1, drop_last=False)
    dataloader_train.append(DataLoader(dataset_train[i], num_workers=1, collate_fn=collater, batch_sampler=sampler))
    dataloader_val.append(DataLoader(dataset_val[i], num_workers=1, collate_fn=collater, batch_sampler=sampler_val))



Loading data...


In [ ]:
retinanets = []
# Create the model
# if DEPTH == 18:
#     retinanet = model.resnet18(num_classes=dataset_train.num_classes(), pretrained=True)
# elif DEPTH == 34:
#     retinanet = model.resnet34(num_classes=dataset_train.num_classes(), pretrained=True)
# elif DEPTH == 50:
#     for i in range(4):
#         retinanets.append(model.resnet50(num_classes=dataset_train[i].num_classes(), pretrained=True))
# elif DEPTH == 101:
#     for i in range(4):
#         retinanets.append(model.resnet101(num_classes=dataset_train[i].num_classes(), pretrained=True)) 
# elif DEPTH == 152:
#     for i in range(4):
#         retinanets.append(model.resnet152(num_classes=dataset_train[i].num_classes(), pretrained=True))
# else:
#     raise ValueError('Unsupported model depth, must be one of 18, 34, 50, 101, 152')
#retinanet = torch.load('weights/RSNA_retinanet_5.pt')
for i in range(2):
    retinanets.append(mymodel.se_resnext50_32x4d(num_classes=dataset_train[i].num_classes()))
optimizer = []
scheduler = []
loss_hist = []
classloss_hist = []
regressloss_hist = []
wholeclassloss_hist = []
for i in range(2):
    retinanets[i] = retinanets[i].cuda()
    #变成并行
    retinanets[i] = torch.nn.DataParallel(retinanets[i]).cuda()
    #训练模式
    retinanets[i].training = True
    #学习率0.00001  
    optimizer.append(optim.Adam(retinanets[i].parameters(), lr=1e-4))
    #如果3个epoch损失没有减少则降低学习率
    scheduler.append(optim.lr_scheduler.ReduceLROnPlateau(optimizer[i], patience=2, verbose=True))
    # TODO 这是干什么 deque是为了高效实现插入和删除操作的双向列表，适合用于队列和栈：这里是定义了一个500的队列
    loss_hist.append(collections.deque(maxlen=500))
    classloss_hist.append(collections.deque(maxlen=500))
    regressloss_hist.append(collections.deque(maxlen=500))
    wholeclassloss_hist.append(collections.deque(maxlen=500))
    print('Num training images: {}'.format(len(dataset_train[i])))
# In[5]:



Num training images: 300
Num training images: 280


In [ ]:
VAL_SIZE = 15
count = [2000,2000]
import traceback
for epoch_num in range(EPOCHS):

    for i in range(2):
        retinanets[i].train()
        retinanets[i].module.freeze_bn()
        epoch_loss = []
        
        for iter_num, data in enumerate(dataloader_train[i]):
            if 1 == 1:
            #try:
                optimizer[i].zero_grad()

                focal_loss, whole_class_loss = retinanets[i]([Variable(data['img'].cuda().float()), Variable(data['annot'].cuda())])
                classification_loss, regression_loss = focal_loss
                classification_loss = classification_loss.mean()
                regression_loss = regression_loss.mean()
                whole_class_loss = whole_class_loss.mean()
                loss = classification_loss + regression_loss + whole_class_loss

                if bool(loss == 0):
                    continue
                #反向传播？
                loss.backward()

                #这是干嘛？？TODO
                torch.nn.utils.clip_grad_norm_(retinanets[i].parameters(), 0.1)

                #这?TODO
                optimizer[i].step()

                loss_hist[i].append(float(loss))
                classloss_hist[i].append(float(classification_loss))
                regressloss_hist[i].append(float(regression_loss))
                wholeclassloss_hist[i].append(float(whole_class_loss))
                epoch_loss.append(float(loss))
                
                print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Whole Class loss: {:1.5f} | Running loss: {:1.5f}'.format(epoch_num, iter_num, np.mean(classloss_hist[i]), np.mean(regressloss_hist[i]), np.mean(wholeclassloss_hist[i]), np.mean(loss_hist[i])))
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(loss_hist[i])]), win='train loss '+str(i), update='append' ,opts={'title':'train loss '+str(i)})
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(classloss_hist[i])]), win='classification loss '+str(i), update='append' ,opts={'title':'classification loss '+str(i)})
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(regressloss_hist[i])]), win='regression loss '+str(i), update='append' ,opts={'title':'regression loss '+str(i)})
                vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([np.mean(wholeclassloss_hist[i])]), win='whole class loss '+str(i), update='append' ,opts={'title':'whole class loss '+str(i)})

                count[i] += 1
                vis.save(['retinanet_seresnextTest'])

                del classification_loss
                del regression_loss
#                 print(count[i])
#                 if count[i] % VAL_SIZE == 0:
#                     print("Evaluating dataset")
#                     retinanets[i].eval()
#                     mAP = csv_eval.evaluate(dataset_val[i],retinanets[i])
#                     vis.line(X=torch.Tensor([count[i]]), Y=torch.Tensor([mAP[0][0]]), win='val mAP '+str(i), update='append' ,opts={'title':'mAP val '+str(i)})
#                     vis.save(['retinanet_seresnextTest'])
#                     #torch.save(retinanets[i].module, 'weights_stage1/{}_seresnext101_{}.pt'.format(i, epoch_num))
#                     retinanets[i].train()
#             except Exception as e:
#                 print(e)
#                 continue


        print("Evaluating dataset")
        retinanets[i].eval()
    #     for index,data in enumerate(dataloader_val):
    #         #data = dataset[index]
    #         scale = data['scale']
    #         # run network
    #         #scores, labels, boxes = retinanet(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
    #         print(Variable(data['img'].cuda()))
    #         scores, labels, boxes = retinanet(Variable(data['img'].cuda()))
    #         #mAP = csv_eval.evaluate(dataset_val,retinanet)
#         try:
        if 1 == 1:
            mAP = csv_eval.evaluate(dataset_val[i],retinanets[i])
            vis.line(X=torch.Tensor([epoch_num]), Y=torch.Tensor([mAP[1][0]]), win='val mAP '+str(i), update='append' ,opts={'title':'mAP val '+str(i)})
            vis.save(['retinanet_seresnextTest'])
#         except Exception as e:
#             print(e)
#             print( 'traceback.print_exc():', traceback.print_exc())
#             continue
        #这一步也看不懂？？TODO 
        scheduler[i].step(np.mean(epoch_loss))

        #torch.save(retinanets[i].module, 'weights_stage1/{}_seresnext101_{}.pt'.format(i, epoch_num))
    #     torch.save(retinanet.module, '{}_retinanet_{}.pt'.format(parser.dataset, epoch_num))

for i in range(2):
    retinanets[i].eval()
    #torch.save(retinanets[i], 'weights_stage1/{}_model_final_seresnext101.pt'.format(i))




flip_x
flip_x
flip_x
scale


/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/data/krf/anaconda/anaconda3/lib/python3.5/site-packages/torch/nn/parallel/_functions.py:58: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch: 0 | Iteration: 0 | Classification loss: 0.12567 | Regression loss: 0.00000 | Whole Class loss: 1.13655 | Running loss: 1.26222
shift
shift
Epoch: 0 | Iteration: 1 | Classification loss: 0.37343 | Regression loss: 0.26745 | Whole Class loss: 1.40960 | Running loss: 2.05048
Epoch: 0 | Iteration: 2 | Classification loss: 0.28346 | Regression loss: 0.17830 | Whole Class loss: 1.27313 | Running loss: 1.73489
shift
Epoch: 0 | Iteration: 3 | Classification loss: 0.23187 | Regression loss: 0.13373 | Whole Class loss: 1.20608 | Running loss: 1.57168
Epoch: 0 | Iteration: 4 | Classification loss: 0.19495 | Regression loss: 0.10698 | Whole Class loss: 1.17726 | Running loss: 1.47919
shift
Epoch: 0 | Iteration: 5 | Classification loss: 0.26068 | Regression loss: 0.18900 | Whole Class loss: 1.14315 | Running loss: 1.59282
shift
scale
Epoch: 0 | Iteration: 6 | Classification loss: 0.22574 | Regression loss: 0.16200 | Whole Class loss: 1.16967 | Running loss: 1.55741
Epoch: 0 | Iteration: 7 | 

Epoch: 0 | Iteration: 58 | Classification loss: 0.20551 | Regression loss: 0.16644 | Whole Class loss: 1.12928 | Running loss: 1.50124
noise
Epoch: 0 | Iteration: 59 | Classification loss: 0.20222 | Regression loss: 0.16367 | Whole Class loss: 1.12649 | Running loss: 1.49238
scale
Epoch: 0 | Iteration: 60 | Classification loss: 0.19903 | Regression loss: 0.16099 | Whole Class loss: 1.12433 | Running loss: 1.48435
noise
Epoch: 0 | Iteration: 61 | Classification loss: 0.19587 | Regression loss: 0.15839 | Whole Class loss: 1.11831 | Running loss: 1.47257
shift
shift
scale
Epoch: 0 | Iteration: 62 | Classification loss: 0.20177 | Regression loss: 0.16390 | Whole Class loss: 1.11655 | Running loss: 1.48222
Epoch: 0 | Iteration: 63 | Classification loss: 0.19867 | Regression loss: 0.16134 | Whole Class loss: 1.11142 | Running loss: 1.47143
flip_x
flip_x
shift
Epoch: 0 | Iteration: 64 | Classification loss: 0.19566 | Regression loss: 0.15886 | Whole Class loss: 1.10648 | Running loss: 1.46100

shift
Epoch: 0 | Iteration: 116 | Classification loss: 0.20003 | Regression loss: 0.16209 | Whole Class loss: 1.13288 | Running loss: 1.49499
scale
Epoch: 0 | Iteration: 117 | Classification loss: 0.19839 | Regression loss: 0.16071 | Whole Class loss: 1.12695 | Running loss: 1.48605
shift
shift
Epoch: 0 | Iteration: 118 | Classification loss: 0.20183 | Regression loss: 0.16318 | Whole Class loss: 1.13746 | Running loss: 1.50247
flip_x
shift
Epoch: 0 | Iteration: 119 | Classification loss: 0.20020 | Regression loss: 0.16182 | Whole Class loss: 1.13115 | Running loss: 1.49317
noise
Epoch: 0 | Iteration: 120 | Classification loss: 0.20316 | Regression loss: 0.16424 | Whole Class loss: 1.14947 | Running loss: 1.51688
flip_x
flip_x
flip_x
Epoch: 0 | Iteration: 121 | Classification loss: 0.20590 | Regression loss: 0.16675 | Whole Class loss: 1.15257 | Running loss: 1.52522
shift
Epoch: 0 | Iteration: 122 | Classification loss: 0.20434 | Regression loss: 0.16540 | Whole Class loss: 1.15256 | 

noise
flip_x
Epoch: 0 | Iteration: 23 | Classification loss: 0.26229 | Regression loss: 0.20387 | Whole Class loss: 1.21101 | Running loss: 1.67716
flip_x
scale
Epoch: 0 | Iteration: 24 | Classification loss: 0.25211 | Regression loss: 0.19572 | Whole Class loss: 1.18660 | Running loss: 1.63442
Epoch: 0 | Iteration: 25 | Classification loss: 0.26643 | Regression loss: 0.20756 | Whole Class loss: 1.21196 | Running loss: 1.68595
flip_x
flip_x
shift
flip_x
Epoch: 0 | Iteration: 26 | Classification loss: 0.25733 | Regression loss: 0.19987 | Whole Class loss: 1.21047 | Running loss: 1.66767
rotate
Epoch: 0 | Iteration: 27 | Classification loss: 0.24893 | Regression loss: 0.19273 | Whole Class loss: 1.19605 | Running loss: 1.63772
shift
Epoch: 0 | Iteration: 28 | Classification loss: 0.24141 | Regression loss: 0.18609 | Whole Class loss: 1.18740 | Running loss: 1.61489
shift
shift
rotate
Epoch: 0 | Iteration: 29 | Classification loss: 0.25248 | Regression loss: 0.19720 | Whole Class loss: 1.

flip_x
noise
Epoch: 0 | Iteration: 81 | Classification loss: 0.20170 | Regression loss: 0.16828 | Whole Class loss: 1.13213 | Running loss: 1.50211
rotate
shift
Epoch: 0 | Iteration: 82 | Classification loss: 0.20706 | Regression loss: 0.17249 | Whole Class loss: 1.13589 | Running loss: 1.51544
shift
flip_x
Epoch: 0 | Iteration: 83 | Classification loss: 0.20460 | Regression loss: 0.17044 | Whole Class loss: 1.13168 | Running loss: 1.50672
scale
Epoch: 0 | Iteration: 84 | Classification loss: 0.20946 | Regression loss: 0.17407 | Whole Class loss: 1.14072 | Running loss: 1.52426
Epoch: 0 | Iteration: 85 | Classification loss: 0.20704 | Regression loss: 0.17205 | Whole Class loss: 1.13251 | Running loss: 1.51160
flip_x
shift
shift
Epoch: 0 | Iteration: 86 | Classification loss: 0.20470 | Regression loss: 0.17007 | Whole Class loss: 1.13954 | Running loss: 1.51431
shift
Epoch: 0 | Iteration: 87 | Classification loss: 0.20241 | Regression loss: 0.16814 | Whole Class loss: 1.13309 | Running

Epoch: 0 | Iteration: 139 | Classification loss: 0.21647 | Regression loss: 0.18066 | Whole Class loss: 1.15040 | Running loss: 1.54753
Evaluating dataset
280/280
mAP:
0: 0.0
1: 0.0
2: 0.0
flip_x
flip_x
flip_x
scale
Epoch: 1 | Iteration: 0 | Classification loss: 0.22062 | Regression loss: 0.18002 | Whole Class loss: 1.14135 | Running loss: 1.54198
Epoch: 1 | Iteration: 1 | Classification loss: 0.21926 | Regression loss: 0.17883 | Whole Class loss: 1.13950 | Running loss: 1.53759
shift
shift
Epoch: 1 | Iteration: 2 | Classification loss: 0.21792 | Regression loss: 0.17766 | Whole Class loss: 1.13742 | Running loss: 1.53301
shift
Epoch: 1 | Iteration: 3 | Classification loss: 0.22002 | Regression loss: 0.17962 | Whole Class loss: 1.13736 | Running loss: 1.53699
Epoch: 1 | Iteration: 4 | Classification loss: 0.22190 | Regression loss: 0.18142 | Whole Class loss: 1.13742 | Running loss: 1.54075
shift
Epoch: 1 | Iteration: 5 | Classification loss: 0.22426 | Regression loss: 0.18359 | Whole 

Epoch: 1 | Iteration: 56 | Classification loss: 0.22103 | Regression loss: 0.18106 | Whole Class loss: 1.13026 | Running loss: 1.53234
noise
Epoch: 1 | Iteration: 57 | Classification loss: 0.22001 | Regression loss: 0.18019 | Whole Class loss: 1.12779 | Running loss: 1.52799
Epoch: 1 | Iteration: 58 | Classification loss: 0.22169 | Regression loss: 0.18161 | Whole Class loss: 1.13055 | Running loss: 1.53385
noise
Epoch: 1 | Iteration: 59 | Classification loss: 0.22068 | Regression loss: 0.18074 | Whole Class loss: 1.13079 | Running loss: 1.53221
scale
Epoch: 1 | Iteration: 60 | Classification loss: 0.21967 | Regression loss: 0.17988 | Whole Class loss: 1.12901 | Running loss: 1.52856
noise
Epoch: 1 | Iteration: 61 | Classification loss: 0.21867 | Regression loss: 0.17904 | Whole Class loss: 1.12814 | Running loss: 1.52585
shift
shift
scale
Epoch: 1 | Iteration: 62 | Classification loss: 0.22035 | Regression loss: 0.18062 | Whole Class loss: 1.12783 | Running loss: 1.52880
Epoch: 1 | It

Epoch: 1 | Iteration: 114 | Classification loss: 0.21478 | Regression loss: 0.17595 | Whole Class loss: 1.12947 | Running loss: 1.52020
noise
Epoch: 1 | Iteration: 115 | Classification loss: 0.21398 | Regression loss: 0.17529 | Whole Class loss: 1.13131 | Running loss: 1.52058
shift
scale
Epoch: 1 | Iteration: 116 | Classification loss: 0.21321 | Regression loss: 0.17464 | Whole Class loss: 1.12979 | Running loss: 1.51764
shift
shift
Epoch: 1 | Iteration: 117 | Classification loss: 0.21451 | Regression loss: 0.17576 | Whole Class loss: 1.13094 | Running loss: 1.52120
flip_x
shift
Epoch: 1 | Iteration: 118 | Classification loss: 0.21374 | Regression loss: 0.17510 | Whole Class loss: 1.13038 | Running loss: 1.51921
noise
Epoch: 1 | Iteration: 119 | Classification loss: 0.21297 | Regression loss: 0.17445 | Whole Class loss: 1.12918 | Running loss: 1.51660
Epoch: 1 | Iteration: 120 | Classification loss: 0.21220 | Regression loss: 0.17381 | Whole Class loss: 1.12741 | Running loss: 1.51343